# Collision Avoidance - ResNet18をTensorRTに変換

学習したPytorchモデルをTensorRTで最適化します。  
``02_train_model_resnet18_JP.ipynb``ノートブックの指示に従って、すでに``best_model_resnet18.pth``を作成していることを想定します。

## 学習済みモデルの読み込み
最初にtorchvisionで提供されている未学習のResNet18モデルを読み込みます。(自前学習した値でモデルを初期化するため、ImageNetで学習済みのモデルである必要がありません。)  
次に、ResNet18モデル構造の全結合層(fully connected layer)を入れ替えて、JetBotの衝突回避モデルで欲しい出力「free」と「blocked」の2種類を得られるモデル構造にします。  

In [7]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch
import torchvision

########################################
# PyTorchで提供されている未学習のResNet18モデルを読込みます。
########################################
model = torchvision.models.resnet18(pretrained=False)

########################################
# モデルの出力層をJetBotの道路走行モデル用に置き換えます。
########################################
model.fc = torch.nn.Linear(512, 4)

########################################
# GPU処理が可能な部分をGPUで処理するように設定します。
# model.eval()は推論実行前に必ず必要になります。
# これはDropoutレイヤーとバッチ正規化レイヤーを学習モードから評価モードに変更します。
# これらは学習時に精度を高めるためにランダムで適用される機能であり、
# 推論時には精度を上げるためにmodel.eval()を実行してこれらの機能を無効にします。
# また、float16型に変更します。
########################################
model = model.cuda().eval().half()

次に、学習済みモデル``best_model_resnet18.pth``から値を読み込み、ResNet18モデルにウェイトを設定します。

In [8]:
########################################
# 未学習のモデルに学習結果の重みづけを読込みます。
########################################
model.load_state_dict(torch.load('./detection.pth'))

<All keys matched successfully>

TensorRTはGPUの利用が必須であり、最初からGPU向けに実装されています。  
そのため、モデルに対するデバイス指定は不要です。  
しかしデータに対しては同じデバイス上に存在する必要があるため、GPUへの転送が必要になります。  

In [9]:
########################################
# このデバイス定義は利用されていません。
########################################
device = torch.device('cuda')

## TensorRTモデルに変換

TorchからTensorRTに変換します。  
TensorRTでの推論を高速化するために、torch2trtを使用してモデルを変換および最適化します。  
詳細については、[torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt)のreadmeを参照してください。

In [10]:
########################################
# 利用するライブラリを読み込みます。
########################################
from torch2trt import torch2trt

########################################
# TensorRT化する際に、サンプルの入力データを渡す必要があります。
# サンプルの入力データを作成します。
# サンプルデータはGPUデバイスにfloat16型で作成します。
########################################
data = torch.zeros((1, 3, 224, 224)).cuda().half()

########################################
# TensorRTモデルを作成します。
########################################
model_trt = torch2trt(model, [data], fp16_mode=True)

TensorRTモデルをファイルに保存します。

In [11]:
########################################
# TensorRTモデルをファイルに保存します。
########################################
torch.save(model_trt.state_dict(), 'detection_trt.pth')

## 次
JetBot本体で学習した場合は、このノートブックを閉じてからJupyter左側にある「Running Terminals and Kernels」を選択して「03_live_demo_resnet18_build_trt_JP.ipynb」の横にある「SHUT DOWN」をクリックしてJupyter Kernelをシャットダウンしてから[04_live_demo_resnet18_trt_JP.ipynb](04_live_demo_resnet18_trt_JP.ipynb)に進んでください。  